In [2]:
import numpy as np
import matplotlib as mp

FLAGS_num_particles = 50
particles_ = list()
prev_x = 0
prev_y = 0
prev_angle = 0

In [3]:
class Particle:
    x = 0
    y = 0
    angle = 0
    weight = 0
    
def magnitude(x, y):
    return Math.sqrt(pow(x, 2) + pow(y, 2))

In [4]:
def initialize(x, y, angle):
    # // The "set_pose" button on the GUI was clicked, or an initialization message
    # // was received from the log Initialize the particles accordingly, eg with
    # // some distribution around the provided location and angle.
    #   map_.Load(map_file);

#   // initialize vector of particles with GetParticles
    for i in range(FLAGS_num_particles):
        p = Particle()
        # // init in Gaussian distribution around loc and angle
        p.x = x + np.random.normal(0.0, 1.0)
        p.y = y + np.random.normal(0.0, 1.0)
        p.angle = angle + np.random.nsormal(0.0, 1.0)
        particles_.append(p);
        
    return x, y, angle

In [5]:
def getlocation():
    x = 0
    y = 0
    angle = 0
    
    # // Compute the best estimate of the robot's location based on the current set
    # // of particles. The computed values must be set to the `loc` and `angle`
    # // variables to return them. Modify the following assignments:

    x_locs = 0.0;
    y_locs = 0.0;
    sines = 0.0;
    cosines = 0.0;

    for i in range(FLAGS_num_particles):
        x_locs += particles_[i].x * particles_[i].weight;
        y_locs += particles_[i].y * particles_[i].weight;
        sines += Math.sin(particles_[i].angle);
        cosines += Math.cos(particles_[i].angle);
    x = x_locs / FLAGS_num_particles # removed x
    y = y_locs / FLAGS_num_particles # removed y
    angle = Math.atan2(sines / FLAGS_num_particles, cosines / FLAGS_num_particles)
    return x, y, angle

In [6]:
def predict(x, y, angle):
    x_hat = x - prev_x
    y_hat = y - prev_y
    theta_hat = angle - prev_angle
    
    # // tunable parameters
    k_1 = 1.0 # //   x,y stddev's   mag(x,y) weight
    k_2 = 1.0 # //   x,y stddev's   mag(theta) weight
    k_3 = 1.0 # // theta stddev's   mag(x,y) weight
    k_4 = 1.0 # // theta stddev's   mag(theta) weight

    # // e_x, e_y drawn from N(0, k1*sqrt(d_x^2 + d_y^2) + k2*||d_theta||)
    # // e_theta drawn from N(0, k3*sqrt(d_x^2 + d_y^2) + k4*||d_theta||)
    # // same distrib except for TUNABLE params k1+k2/k3+k4
    xy_stddev = k_1*magnitude(x_hat, y_hat) + k_2*abs(theta_hat);
    theta_stddev = k_3*magnitude(x_hat, y_hat) + k_4*abs(theta_hat);
    # // update particles
    for i in range(FLAGS_num_particles):
        e_x = np.random.normal(0.0, xy_stddev)
        e_y = np.random.normal(0.0, xy_stddev)
        e_theta = np.random.normal(0.0, theta_stddev)
        particles_[i].x += x_hat + e_x;
        particles_[i].y += y_hat + e_y;
        particles_[i].angle += theta_hat + e_theta;

In [7]:
# // Returns the value of the 2D cross product between v1 and v2.
# template <typename T>
# T Cross(const Eigen::Matrix<T, 2, 1>& v1, const Eigen::Matrix<T, 2, 1>& v2) {
#   return (v1.x() * v2.y() - v2.x() * v1.y());
# }

def cross(v1x, v1y, v2x, v2y):
    return v1x * v2y - v2x * v1y

def intersection(p0x, p0y, p1x, p1y, p2x, p2y, p3x, p3y):
    # // Bounding-box broad phase check.
    if min(p0x, p1x) > max(p2x, p3x): return False
    if max(p0x, p1x) < min(p2x, p3x): return False
    if min(p0x, p1x) > max(p2x, p3x): return False
    if max(p0x, p1x) < min(p2x, p3x): return False
    # // Narrow-phase check.
    d1x = p1x - p0x
    d1y = p1y - p0y
    d2x = p3x - p2x
    d2y = p3y - p2y
    # if (Cross<T>(d1, p3 - p0) * Cross<T>(d1, p2 - p0) > 0.0): return False
    if cross(d1x, d1y, (p3x - p0x), (p3y - p0y)) * cross(d1x, d1y, (p2x - p0x), (p2y - p0y)): return False
    # if (Cross<T>(d2, p1 - p2) * Cross<T>(d2, p0 - p2) > 0.0): return False
    if cross(d2x, d2y, (p1x - p2x), (p1y - p2y)) * cross(d2x, d2y, (p0x - p2x), (p0y - p2y)): return False
    # // Okay, the line segments definitely intersect.
    # const T d = Cross<T>(d2, -d1);
    d = cross(d2x, d2y, d1x * -1, d1y * -1)

    if (d == 0): return False

    # const T tb = Cross<T>(p0 -p2, p0-p1) / d;
    tb = cross(p0x - p2x, p0y - p2y, p0x - p1x, p0y - p1y)
    # *intersection = p2 + tb * d2;
    intersection_x = p2x + tb * d2x
    intersection_y = p2y + tb * d2y
    return True, intersection_x, intersection_y;

In [8]:
def gppc(x, y, angle, num_ranges, map_lines, angle_min, angle_max, range_min, range_max, scan):
    laser_x = x + 0
    laser_y = y + 0.2
    # // Compute what the predicted point cloud would be, if the car was at the pose
    # // loc, angle, with the sensor characteristics defined by the provided
    # // parameters.
    # // This is NOT the motion model predict step: it is the prediction of the
    # // expected observations, to be used for the update step.

    # scan.resize(num_ranges);
    # // iterate through scan to set points for each ray
    for i in range(num_ranges):
        # // iterate through map to check for collisions
        for j in range(len(map_lines)):
            # // to check for collisions, construct a line2f from range_min to range_max, in the direction of the ray, centered around laser pose
            map_line = map_lines[j]
            # // need to use math to calculate endpoint of the line based on range_max. treat laser location as point 0
            # // 10 is a magic number rn describing the angle increment. we should tune that (and by extension num_ranges)
            alpha = angle_min + 10*i;
            # // the range max point
            rm_x = range_max * Math.sin(alpha)
            rm_y = range_max * Math.cos(alpha)
            # my_line(laser_loc.x(), laser_loc.y(), rm_pt.x(), rm_pt.y())
            # // check for intersection with this line and the map line
            intersects, intersection_x, intersection_y = intersection(map_line, laser_x, laser_y, rm_x, rm_y) # // Return variable
            if (intersects):
                # // if intersection exists, "first" collision wins
                scan[i][0] = intersection_x
                scan[i][1] = intersection_y
                break
            else:
                # // else if no collision, set scan[i] to the point at range_max
                scan[i][0] = rm_x
                scan[i][1] = rm_y
                
    return scan

In [9]:
def update(ranges, map_lines, angle_max, angle_min, range_min, range_max, particle):
    num_ranges = (angle_max - angle_min) / 10;
    scan = list()
    for i in range(num_ranges):
        scan.append(list())

    x, y, angle = getlocation(); # // TODO: doubt this is correct -sun
    # // use map-relative location actually
    # // num_ranges should equal something like (angle_max - angle_min) / 10, so every 10 degrees we use the lidar range
    gppc(x, y, angle, num_ranges, map_lines, angle_min, angle_max, range_min, range_max, scan)
    
    
    # // compare particle observation to prediction
    # // ranges holds our observation s, scan_ptr (kinda) holds our prediction s_hat
    log_lik = 0.0
    # // tunable param: sd_squared
    sd_squared = 0.0025 # // TODO: did the slides say to start at 0.05 std dev? unsure.
    # // robustification will be on 14 - Expecting The Unexpected slide 28
    # for (size_t i = 0; i < scan_ptr.size(); ++i) {
    for i in range(num_ranges):
        # // s_hat is (dist btwn laser and scan[i] points) - range_min
        # // TUNABLE: check if this should be sqnorm instead of norm if particle filter is slow
        # // TODO: check if I need to subtract laser_loc.x and .y from here (and also figure out how to calc that)
        s_hat_dist = Math.sqrt(pow(scan[i][0], 2) + pow(scan[i][1], 2))
        s_hat = s_hat_dist - range_min
        # // s is the range, aka dist from laser to endpoint of observed
        s = ranges[i * 10] # // TUNABLE: every 10th laser?
        log_lik -= Math.ln(pow((s_hat - s), 2) / sd_squared) # NOTE: TODO: maybe not -=? += instead?
        
    particle.weight = log_lik;